In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision

In [2]:
import torchvision.transforms as transforms
from vl_datasets.sentry import v1_sentry_handler, vl_capture_log_debug_state


train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

valid_transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [3]:
from vl_datasets import VLImageNet
train_dataset = VLImageNet('../../imagenet-1k/archives', split='train', exclude_csv='../../imagenet-1k/archives/ImageNet-1K_images_issue_file_list.csv', transform=train_transform)
valid_dataset = VLImageNet('../../imagenet-1k/archives', split='val', exclude_csv='../../imagenet-1k/archives/ImageNet-1K_images_issue_file_list.csv', transform=valid_transform)

Using provided CSV file: ../../imagenet-1k/archives/ImageNet-1K_images_issue_file_list.csv
Using provided CSV file: ../../imagenet-1k/archives/ImageNet-1K_images_issue_file_list.csv


In [4]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=256, shuffle=True)

In [5]:
model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

For the purpose of demonstration, we will be training only on 1% of the total images. 

In [7]:
# Define the portion of the dataset you want to train on
subset_ratio = 0.01  # Train on 1% of the dataset
subset_size = int(subset_ratio * len(train_loader.dataset))
subset_sampler = torch.utils.data.SubsetRandomSampler(range(subset_size))
subset_loader = torch.utils.data.DataLoader(train_loader.dataset, sampler=subset_sampler)

In [8]:
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(subset_loader): # to train on the full dataset replace subset_loader with train_loader
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {running_loss/len(train_loader)}")

Using device: cpu
Epoch 1 - Loss: 6.119085155736327
Epoch 2 - Loss: 5.854837443867154
Epoch 3 - Loss: 4.923516098672809
Epoch 4 - Loss: 4.275343816862248
Epoch 5 - Loss: 3.8568021098517646


In [9]:
correct = 0
total = 0
with torch.no_grad():
    for data in valid_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}")

Accuracy: 0.19878699365098684
